In [2]:
import nest_asyncio
nest_asyncio.apply()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import tensorflow_federated as tff

import numpy as np

## Showcase Client Federated Data & Processing

This is aimed to provide an insight for federated data and operations of federated data, i.e., data that lie in clients and processing that happens on clients (locally).

Let's start with an example of TF (non-federated, just local) code that takes a dataset and does something with it, say add numbers:

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def process_data(ds):
    return ds.reduce(np.float32(0), lambda x, y: x + y)

This code takes a dataset of integer numbers at input, and returns a single integer with the sum at output.

You can confirm this by lookin at the type signature, like this:

In [ ]:
str(process_data.type_signature)

So, `process_data` takes a set of integers, and returns an integer.

Now, using TFF's federated operators we can create a federated computation that does this **on multiple clients**, like this:

In [ ]:
CLIENT_DATA_TYPE = tff.type_at_clients(tff.SequenceType(tf.float32))

In [ ]:
str(CLIENT_DATA_TYPE)

In [ ]:
@tff.federated_computation(CLIENT_DATA_TYPE)
def process_data_on_clients(federated_ds):
    return tff.federated_map(process_data, federated_ds)

Let's look at the type signature of this computation

In [ ]:
str(process_data_on_clients.type_signature)

This means `process_data_on_clients` takes a federated set of integers (one set per client), and returns a federated integer (one integer with the sum on each client).

What happens in the above is that, the TF logic in `process_data` will be **executed once on each client**. This is how the `federated_map` operator works.

Here's some TF code that creates a single dataset with integers, say you supply an integer $n$ and want to create a dataset with numbers from 1 up to n, i..e, $\{1, 2, ..., n\}$:

In [ ]:

@tff.tf_computation(tf.float32)
def make_data(n):
    return tf.data.Dataset \
        .range(tf.cast(n, tf.int64)) \
        .map(lambda x: tf.cast(x + 1, tf.float32))

This is obviously a silly example, we could do something more along the lines of what we need (e.g., read data from a file specified by a name, etc.).

And here's what its type signature looks like:

In [ ]:
str(make_data.type_signature)

We can see the similarity to `process_data`.

And, just like with processing data, here's now we can make data on all clients by using the `federated_map` operator:

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def make_data_on_clients(federated_n):
    return tff.federated_map(make_data, federated_n)

This is the type signature:

In [ ]:
str(make_data_on_clients.type_signature)

Great, so `make_data_on_clients` takes a federated integer (that tells us how many data items to produce on each client), and returns a federated dataset, just like what `process_data_on_clients` wants.

We can check that the two work together as intended:

In [ ]:
federated_n = [2., 3., 4.]
federated_ds = make_data_on_clients(federated_n)
result = process_data_on_clients(federated_ds)
result

We got the sums 1+2, 1+2+3, and 1+2+3+4 on the 3 clients involved in this computation (note there were 3 numbers in the federated integer above, so there are 3 clients here)

In out silly example, dataset construction logic used range, but it can be anything. For example, you could load data on each client from the same local file `my_data`, or using a custom TF op, or by whatever other means. Just like in our example, you we pass parameters to that function to give you more centralized control (similarly to whatever did above with the federated integer).

## Geometric Monitoring Approach

For federated aggregations like tff.federated_aggregate, TFF also has a mechanism for early stopping. Specifically, when a single client contributes a value of the aggregate that makes it impossible for any other client's contribution to affect the final result, then TFF stops waiting for other clients to contribute their values, and returns the final result early. This can happen in the case where any client returns True, and exceeded becomes True.

In [5]:


CLIENT_DATA_TYPE = tff.type_at_clients(tff.SequenceType(tf.float32))


@tf.function
def compute_sum(ds):
    exceeded = False
    s = 0.
    for i, x in enumerate(ds):
        if i > 50000:
            exceeded = True
            break
        s += x
    return s, exceeded


@tff.tf_computation(tff.SequenceType(tf.float32))
def process_data(ds):
    return compute_sum(ds)
    #return ds.reduce(np.float32(0), lambda x, y: x + y)


@tff.tf_computation(tf.float32)
def make_data(n):
    return tf.data.Dataset \
        .range(tf.cast(n, tf.int64)) \
        .map(lambda x: tf.cast(x + 1, tf.float32))


@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def make_data_on_clients(federated_n):
    return tff.federated_map(make_data, federated_n)


@tff.tf_computation(tf.bool, tf.bool)
def bool_or(x,y):
    return tf.reduce_any([x,y])


@tff.federated_computation(CLIENT_DATA_TYPE)
def test_round(federated_ds):
    # Sum for each client
    client_sums, client_conds = tff.federated_map(process_data, federated_ds)
    
    # Check if any client exceeded the limit
    exceeded = tff.federated_aggregate(
        client_conds,
        zero=False,
        accumulate=bool_or,
        merge=bool_or,
        report=tff.tf_computation(lambda b: b),
    )
    
    # Compute the mean of the client sums
    mean_sum = tff.federated_mean(client_sums)
    
    return mean_sum, exceeded

In [ ]:

federated_n = [60000., 60000., 30000.]
federated_ds = make_data_on_clients(federated_n)

In [ ]:

res, exceeded = test_round(federated_ds)

In [ ]:
res

In [ ]:
exceeded

In [42]:


CLIENT_DATA_TYPE = tff.type_at_clients(tff.SequenceType(tf.float32))


@tf.function
def compute_sum(ds):
    s = 0.
    for x in ds:
        s += x
    return s


@tff.tf_computation(tff.SequenceType(tf.float32))
def process_data(ds):
    return compute_sum(ds)
    #return ds.reduce(np.float32(0), lambda x, y: x + y)


@tff.tf_computation(tf.float32)
def make_data(n):
    return tf.data.Dataset \
        .range(tf.cast(n, tf.int64)) \
        .map(lambda x: tf.cast(x + 1, tf.float32))


@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def make_data_on_clients(federated_n):
    return tff.federated_map(make_data, federated_n)


@tff.federated_computation(CLIENT_DATA_TYPE)
def test_round(federated_ds):
    
    # Sum for each client
    client_sums = tff.federated_map(process_data, federated_ds)
    
    # Compute the mean of the client sums
    mean_sum = tff.federated_mean(client_sums)
    
    return mean_sum

In [43]:

federated_n = [60000., 60000., 30000.]
federated_ds = make_data_on_clients(federated_n)

In [44]:
res = test_round(federated_ds)

In [45]:
res

60743807000.0

In [29]:
res

1350002700.0

In [40]:
res

60743807000.0

In [16]:
tff.federated_value

<function tensorflow_federated.python.core.impl.federated_context.intrinsics.federated_value(value, placement)>

In [46]:
control_var_type = tff.type_at_server(tf.bool)
control_var = tff.federated_value(False, control_var_type)

ContextError: `tff.Value`s should only be used in contexts which can bind references, generally a `FederatedComputationContext`. Attempted to bind the result of wrapping a computation as a value in a context <tensorflow_federated.python.core.impl.execution_contexts.sync_execution_context.SyncExecutionContext object at 0x7fe11ce5efd0> of type <class 'tensorflow_federated.python.core.impl.execution_contexts.sync_execution_context.SyncExecutionContext'>.

SAD https://stackoverflow.com/questions/66265109/federated-learning-in-tensorflow-federated-is-there-any-way-to-apply-early-stop